<a href="https://colab.research.google.com/github/Kuroin/AI-Projects/blob/main/sqli_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier


dt1 = pd.read_csv("sqliv2.csv",encoding="utf-16-le")
dt2 = pd.read_csv("SQLiV3.csv",encoding="utf-16-le")


dt = pd.concat([dt1,dt2])

X = dt["Sentence"]
y = dt["Label"]

vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()


X.shape = (33761,64,64,1)
X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

trainX=X_train.copy()
trainX.shape=(X_train.shape[0],trainX.shape[1]*trainX.shape[2])
testX=X_test.copy()
testX.shape=(testX.shape[0],testX.shape[1]*testX.shape[2])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 256)        0

In [ ]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128)

Epoch 1/10
211/211 [==============================] - 455s 2s/step - loss: 0.1921 - accuracy: 0.9381 - val_loss: 0.1313 - val_accuracy: 0.9606
Epoch 2/10
211/211 [==============================] - 451s 2s/step - loss: 0.1318 - accuracy: 0.9628 - val_loss: 0.1243 - val_accuracy: 0.9664
Epoch 3/10
211/211 [==============================] - 449s 2s/step - loss: 0.1243 - accuracy: 0.9654 - val_loss: 0.1185 - val_accuracy: 0.9661
Epoch 4/10
211/211 [==============================] - 449s 2s/step - loss: 0.1190 - accuracy: 0.9663 - val_loss: 0.1149 - val_accuracy: 0.9682
Epoch 5/10
211/211 [==============================] - 450s 2s/step - loss: 0.1174 - accuracy: 0.9677 - val_loss: 0.1182 - val_accuracy: 0.9683
Epoch 6/10
211/211 [==============================] - 451s 2s/step - loss: 0.1183 - accuracy: 0.9677 - val_loss: 0.1158 - val_accuracy: 0.9668
Epoch 7/10
211/211 [==============================] - 450s 2s/step - loss: 0.1135 - accuracy: 0.9691 - val_loss: 0.1147 - val_accuracy: 0.9682

In [ ]:
def response(input_val): 
  input_val = [input_val]
  input_val=vectorizer.transform(input_val).toarray()
  input_val.shape=(1,64,64,1)
  result=model.predict(input_val)
  if result>0.5:
    return("Zararlı sql kodu olabilir !!!")
  elif result<=0.5:
    return("Güvenli gözüküyor.")

In [ ]:
while True:
    input_value = input(">>>")
    if input_value =="quit":
      break
    else:
      print(response(input_value))

>>>SELECT * FROM Users WHERE UserId = 105 OR 1=1;
Zararlı sql kodu olabilir !!!
>>>SELECT UserId, Name, Password FROM Users WHERE UserId = 105 or 1=1;
Zararlı sql kodu olabilir !!!
>>>SELECT * FROM Users WHERE Name ="" or ""="" AND Pass ="" or ""=""
Zararlı sql kodu olabilir !!!
>>>erciyes
Güvenli gözüküyor.
>>>Atakan
Güvenli gözüküyor.
>>>quit
